In [1]:
!pip install google-generativeai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2onnx 1.16.1 requires protobuf~=3.20, but you have protobuf 4.25.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/165.0 kB ? eta -:--:--
   --------- ----------------------------- 41.0/165.0 kB 991.0 kB/s eta 0:00:01
   ----------------------------- ---------- 122.9/165.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 165.0/165.0 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/725.4 kB ? eta -:--:--
   ---------------------------- ---------- 522.2/725.4 kB 10.9 MB/s eta 0:00:01
   --------------------------------------- 725.4/725.4 kB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
   --------------------------------------- 413.4/413.4 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:

import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  system_instruction="You are well experienced dietician and nutritionist. You will be given the Gae, height, weight and dietary precautions and goals of the person. \nYou will be given a product and its ingredients select one from each category based on the ingredients and health goals of the person.\nCategory 1: how frequently can i consume it based on my weight goals:\n1. daily consumption\n2. weekly consumption\n3. monthly consumption\nCategory 2: and as per my diet is this food :\n1. nutritional\n2. recreations\n3.  or regular consumption for me\n\nCategory 3: If the food product follows any of the following:\n1. Organic\n2. Sustainable: \n3. Supports Small Businesses\n4. Animal Cruelty-Free: \n\nCategory 4: flag alergens if they are related to my allergy otherwise don't flag a warning\nCategory 5: you need to also flag diet type match this with the input of user info and flag if it is suitable for their dietary type or restrictions.\nCategory 6: Alert if there is a higher presence of nutrients desired in low qty (fats, sugar, sodium, calories)\nYour final answer should only contain tags among one option of each category based on product nutrition analysis.\nIf the answer is None just drom that category output.\nFinally if the 4 out of 6 categories are suitable for this user add that as highly recommended product.\nYour next input will be the info of the person, the name of the product, and screeshot of the ingredients\n",
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("", mime_type="image/png"),
]

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
        "Person info:  \n \t\nI  am a man 53 years old weight 90 kgs, i am 5'0 ft,i am vegan and i want to loose 10kgs.\nseverely allergic to gluten.\n\nProduct info: Pringles Potato Chips Desi Masala Tadka Flavor Pack of 4, Crispy Snack for Game Nights Chips  (4 x 102 g)\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Category 1: monthly consumption\nCategory 2: recreation consumption\nCategory 3: None\nCategory 4: Gluten\nCategory 5: Not Suitable\nCategory 6: High in Fat\nHighly recommended product: No\n",
      ],
    },
  ]
)

response = chat_session.send_message("INSERT_INPUT_HERE")

print(response.text)